In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
from datetime import timedelta
import dataframe_image as dfi
import matplotlib.pyplot as plt
import sklearn.metrics as mtr
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import pickle


pd.options.display.max_colwidth = 200

In [2]:
df = pd.read_csv("../Bases/Cleaned/rota32893_weather.csv")
df

,data_partida,tempo_viagem,dia_semana,partidaTimeStamp,qtdDiasAno,tipo_dia,turno_dia,hora_dia,Precipitacao,Umidade,Vento,Temperatura
0,2019-09-30 23:01:36,1702,1,1569895296,273,1,4,23,0.0,78,1.7,17.25
1,2019-09-30 22:40:17,1671,1,1569894017,273,1,4,22,0.0,78,1.7,17.25
2,2019-09-30 22:30:04,1453,1,1569893404,273,1,4,22,0.0,78,1.7,17.25
3,2019-09-30 21:50:20,1867,1,1569891020,273,1,4,21,0.0,74,2.4,17.75
4,2019-09-30 21:38:06,1590,1,1569890286,273,1,4,21,0.0,74,2.4,17.75
...,...,...,...,...,...,...,...,...,...,...,...,...
5267,2019-07-01 07:27:18,2123,1,1561976838,182,1,2,7,0.0,63,2.1,19.05
5268,2019-07-01 07:06:04,2912,1,1561975564,182,1,2,7,0.0,63,2.1,19.05
5269,2019-07-01 06:47:57,2163,1,1561974477,182,1,2,6,0.0,63,2.1,19.05
5270,2019-07-01 06:05:57,2343,1,1561971957,182,1,2,6,0.0,61,2.0,18.80


In [3]:
#Separação teste e treino
shape = int(df.shape[0] * 0.3)
df_teste = df[:shape]
df_treino = df[shape:]

In [4]:
best_models = None
best_metric = None
best_cenario = [0,0,0,0]

In [5]:
for cenario in range(0, 4):
        if cenario == 0:
                x_teste = df_teste[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia"]]
                y_teste = df_teste["tempo_viagem"]
                x_treino = df_treino[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia"]]
                y_treino = df_treino["tempo_viagem"]

        elif cenario == 1:
                x_teste = df_teste[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "turno_dia"]]
                y_teste = df_teste["tempo_viagem"]
                x_treino = df_treino[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "turno_dia"]]
                y_treino = df_treino["tempo_viagem"]
        
        elif cenario == 2:
                x_teste = df_teste[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "qtdDiasAno"]]
                y_teste = df_teste["tempo_viagem"]
                x_treino = df_treino[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "qtdDiasAno"]]
                y_treino = df_treino["tempo_viagem"]
        
        else:
                x_teste = df_teste[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "turno_dia", "qtdDiasAno"]]
                y_teste = df_teste["tempo_viagem"]
                x_treino = df_treino[["partidaTimeStamp", "dia_semana", "tipo_dia", "hora_dia", "turno_dia", "qtdDiasAno"]]
                y_treino = df_treino["tempo_viagem"]
        

        # Hyperparâmetros do XGBoosting
        params_xgb = {
                "n_estimators": list(range(100, 1100, 100)), #Number of gradient boosted trees. Equivalent to number of boosting rounds
                "max_depth": list(range(2, 15)),#Maximum tree depth for base learners.
                "min_child_weight": list(range(1, 11)),#Minimum sum of instance weight(hessian) needed in a child.
                "learning_rate": [0.3, 0.2, 0.1, 0.05, 0.01, 0.005],#Boosting learning rate (xgb’s “eta”)
                "gamma": np.arange(0, 0.7, 0.1)#Minimum loss reduction required to make a further partition on a leaf node of the tree.
                }

        # Hyperparâmetros do Random Forest
        params_rf = {
                "n_estimators": list(range(100, 1100, 100)),#The number of trees in the forest.
                "bootstrap": [True, False],#Whether bootstrap samples are used when building trees. If False, the whole dataset is used to build each tree.
                "max_depth": list(range(2, 15)),#The maximum depth of the tree.
                "max_features": [1.0, "sqrt", "log2"],#The number of features to consider when looking for the best split:
                "min_samples_leaf": list(range(1, 11)),#The minimum number of samples required to split an internal node
                "min_samples_split": list(range(2, 11)),#The minimum number of samples required to be at a leaf node.
                }

        # Hyperparâmetros do SVR
        params_svr = {
                "C": [0.1, 1, 10, 100, 1000], #Regularization parameter. The strength of the regularization is inversely proportional to C
                "gamma": ["scale", "auto"],#Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’.
                }

        #Linear Regression não tem hyper parametrização


        #Criação e fit do modelo XGB
        modelo_xgb = xgb.XGBRegressor(early_stop_rounds = 100)
        xgb_rand_search = RandomizedSearchCV(modelo_xgb, params_xgb, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
        xgb_rand_search.fit(x_treino, y_treino)
        modelo_xgb = xgb_rand_search.best_estimator_

        modelo_xgb.fit(x_treino, y_treino, eval_set = [(x_treino, y_treino)])


        #Criação e fit do modelo Random Forest
        modelo_rf = RandomForestRegressor()
        rf_rand_search = RandomizedSearchCV(modelo_rf, params_rf, scoring="neg_mean_squared_error", n_iter=40, verbose=True, cv=10, n_jobs=-1, random_state=123)
        rf_rand_search.fit(x_treino, y_treino)
        modelo_rf = rf_rand_search.best_estimator_

        modelo_rf.fit(x_treino, y_treino)


        #Criação e fit do modelo SVR
        modelo_svr = SVR(kernel = 'rbf')
        svr_rand_search = RandomizedSearchCV(modelo_svr, params_svr, scoring="neg_mean_squared_error", n_iter=10, verbose=True, cv=10, n_jobs=-1, random_state=123)
        svr_rand_search.fit(x_treino, y_treino)
        modelo_svr = svr_rand_search.best_estimator_

        modelo_svr.fit(x_treino, y_treino)


        #Criação e fit do modelo Linear Regression
        modelo_lr = LinearRegression()
        modelo_lr.fit(x_treino, y_treino)

        df_teste["predicao_xgb"] = modelo_xgb.predict(x_teste)
        df_teste["predicao_lr"] = modelo_lr.predict(x_teste)
        df_teste["predicao_rf"] = modelo_rf.predict(x_teste)
        df_teste["predicao_svr"] = modelo_svr.predict(x_teste)

        df_teste.to_csv(f"../Bases/Predictions/rota32893_predictions_cen_{cenario}.csv",index=False)

        RMSE_xgb = mtr.mean_squared_error(df_teste["tempo_viagem"], df_teste["predicao_xgb"], squared=False) 
        RMSE_rf = mtr.mean_squared_error(df_teste["tempo_viagem"], df_teste["predicao_rf"], squared=False)       
        RMSE_svr = mtr.mean_squared_error(df_teste["tempo_viagem"], df_teste["predicao_svr"], squared=False) 
        RMSE_lr = mtr.mean_squared_error(df_teste["tempo_viagem"], df_teste["predicao_lr"], squared=False) 

        models = [modelo_xgb, modelo_rf, modelo_svr, modelo_lr]
        metrics = [RMSE_xgb,RMSE_rf,RMSE_svr,RMSE_lr]

        if best_models is None:
                best_models = models
                best_metrics = metrics
        else:
                for i in range(4):
                        if best_metrics[i] > metrics[i]:
                               best_models[i] = models[i]
                               best_metrics[i] = metrics[i]
                               best_cenario[i] = cenario


Fitting 10 folds for each of 40 candidates, totalling 400 fits
[10:37:28] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[10:37:29] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:2895.44221
[1]	validation_0-rmse:2882.31779
[2]	validation_0-rm

C:\Users\Thiago\AppData\Local\Temp\ipykernel_7244\2261576918.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_xgb"] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_7244\2261576918.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_lr"] = modelo_lr.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_7244\2261576918.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

Fitting 10 folds for each of 40 candidates, totalling 400 fits
[10:39:42] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[10:39:43] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:2895.44221
[1]	validation_0-rmse:2882.31779
[2]	validation_0-rm

C:\Users\Thiago\AppData\Local\Temp\ipykernel_7244\2261576918.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_xgb"] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_7244\2261576918.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_lr"] = modelo_lr.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_7244\2261576918.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

Fitting 10 folds for each of 40 candidates, totalling 400 fits
[10:41:55] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[10:41:55] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:2895.44221
[1]	validation_0-rmse:2882.31779
[2]	validation_0-rm

C:\Users\Thiago\AppData\Local\Temp\ipykernel_7244\2261576918.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_xgb"] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_7244\2261576918.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_lr"] = modelo_lr.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_7244\2261576918.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

Fitting 10 folds for each of 40 candidates, totalling 400 fits
[10:44:13] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[10:44:14] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stop_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-rmse:2895.44221
[1]	validation_0-rmse:2882.31779
[2]	validation_0-rm

C:\Users\Thiago\AppData\Local\Temp\ipykernel_7244\2261576918.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_xgb"] = modelo_xgb.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_7244\2261576918.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teste["predicao_lr"] = modelo_lr.predict(x_teste)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_7244\2261576918.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

In [7]:
models_name = ['xgb', 'rf', 'svr', 'lr']
for i in range(4):
    with open (f'ModelsF1/rota32893_{models_name[i]}_cen_{best_cenario[i]}.pkl', 'wb') as m:
        pickle.dump(best_models[i], m)